# IV Calculator

In [64]:
import yfinance as yf
import numpy as np
import pandas as pd
from scipy.stats import norm
from scipy.optimize import minimize_scalar

In [71]:
# option data
symbol = "^SPX"
ticker = yf.Ticker(symbol)
ticker.options
exp_date = ticker.options[25]
option_chain = ticker.option_chain(exp_date)
calls = option_chain.calls
# Filter out SPXW options
calls = calls[~calls['contractSymbol'].str.contains('SPXW')]
calls = calls[calls["openInterest"] > 500]
calls = calls[calls["volume"] > 100]

In [73]:
#stock data
symbol = "^SPX"
data = yf.download(symbol, start="2023-01-01", interval="1d")["Close"]
calls["stockPrice"] = data[-1]
#r 
RF_rate = yf.download("^IRX", start="2022-01-01", interval="1d")["Close"]
calls["RF_rate"] = RF_rate[-1]/100
#Maturity
calls["Maturity"] = 45/252
calls

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,stockPrice,RF_rate,Maturity
96,SPX231117C04000000,2023-09-15 18:59:40+00:00,4000.0,497.38,486.00,500.60,-46.440002,-8.539591,1056.0,28071,0.285427,True,REGULAR,USD,4450.319824,0.05298,0.178571
198,SPX231117C04550000,2023-09-15 19:37:36+00:00,4550.0,56.25,54.30,56.70,-24.029999,-29.932734,419.0,6115,0.132218,False,REGULAR,USD,4450.319824,0.05298,0.178571
203,SPX231117C04575000,2023-09-15 19:59:57+00:00,4575.0,44.60,44.60,45.80,-22.989998,-34.013905,205.0,1099,0.127629,False,REGULAR,USD,4450.319824,0.05298,0.178571
208,SPX231117C04600000,2023-09-15 19:57:25+00:00,4600.0,36.00,34.70,37.50,-19.270000,-34.865208,163.0,8623,0.125302,False,REGULAR,USD,4450.319824,0.05298,0.178571
218,SPX231117C04650000,2023-09-15 20:05:29+00:00,4650.0,22.15,22.10,23.00,-13.250002,-37.429382,146.0,9486,0.118787,False,REGULAR,USD,4450.319824,0.05298,0.178571
223,SPX231117C04675000,2023-09-15 17:52:43+00:00,4675.0,17.95,17.00,17.80,-10.250000,-36.347520,355.0,3630,0.116517,False,REGULAR,USD,4450.319824,0.05298,0.178571
236,SPX231117C04750000,2023-09-15 19:51:08+00:00,4750.0,7.57,7.30,7.80,-6.280000,-45.342960,428.0,4504,0.111860,False,REGULAR,USD,4450.319824,0.05298,0.178571
242,SPX231117C04800000,2023-09-15 20:14:12+00:00,4800.0,4.40,4.10,4.50,-2.700000,-38.028164,188.0,7442,0.110971,False,REGULAR,USD,4450.319824,0.05298,0.178571
253,SPX231117C04890000,2023-09-14 19:05:13+00:00,4890.0,2.86,1.50,1.75,0.000000,0.000000,519.0,904,0.112253,False,REGULAR,USD,4450.319824,0.05298,0.178571
261,SPX231117C05000000,2023-09-15 20:06:50+00:00,5000.0,0.62,0.55,0.70,-0.300000,-32.608696,1085.0,30431,0.118478,False,REGULAR,USD,4450.319824,0.05298,0.178571


In [74]:
def black_scholes(option_type, S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == 'call':
        option_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    else:
        option_price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)

    return option_price

def implied_volatility(option_price, option_type, S, K, T, r):
    def error_function(sigma):
        return abs(black_scholes(option_type, S, K, T, r, sigma) - option_price)

    result = minimize_scalar(error_function, bounds=(0, 2), method='bounded')
    implied_vol = result.x
    return implied_vol

def calculate_implied_volatility(row):
    option_price = row["lastPrice"]
    option_type = 'call'
    S = row["stockPrice"]
    K = row["strike"]
    T = row["Maturity"]
    r = row["RF_rate"]
    
    implied_vol = implied_volatility(option_price, option_type, S, K, T, r)
    return implied_vol

def main():
    # Assuming calls is a DataFrame with the option data
    calls["IV calc"] = calls.apply(calculate_implied_volatility, axis=1)

if __name__ == "__main__":
    main()

In [79]:
df = pd.DataFrame({'Strike Price': calls["strike"], 'OptionPrice': calls["lastPrice"], "Implied Volatility": calls["IV calc"]})
df

,Strike Price,OptionPrice,Implied Volatility
96,4000.0,497.38,0.182671
198,4550.0,56.25,0.108001
203,4575.0,44.60,0.104044
208,4600.0,36.00,0.102622
218,4650.0,22.15,0.099536
223,4675.0,17.95,0.100229
236,4750.0,7.57,0.097158
242,4800.0,4.40,0.097782
253,4890.0,2.86,0.109703
261,5000.0,0.62,0.106611
